In [20]:
import bs4 as bs
import urllib.request
import pandas as pd
import numpy as np
import requests
import csv
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)



In [9]:

def get_soup(page):

    if page == "data":
        
        url = 'http://www.football-data.co.uk/data'
        source = urllib.request.urlopen(url)
        return(bs.BeautifulSoup(source,'lxml'))
    
    else:   
        
        source = urllib.request.urlopen('http://www.football-data.co.uk/data')
        soup = bs.BeautifulSoup(source,'lxml')
        
        for i in soup.find(text="Extra Leagues").findNext('table').find_all('a'):
            
            link = str(i.get("href"))
            
            if link.find(page) !=-1:
                
                url = 'http://www.football-data.co.uk/'+link
                source1 = urllib.request.urlopen(url)
                soup1 = bs.BeautifulSoup(source1,'lxml')
                return(soup1)
            
            else:
                continue
         
        for i in soup.find(text="Main Leagues").findNext('table').find_all('a'):
            
            link = str(i.get("href"))
            
            if link.find(page.lower()) !=-1:
                
                url = 'http://www.football-data.co.uk/'+link
                source2 = urllib.request.urlopen(url)
                soup2 = bs.BeautifulSoup(source2,'lxml')
                return(soup2)
            
            else:
                
                continue
            
        return("no-link")
        
#Return an list of all the countries and leagus which avaliable at the web
def get_country(soup):
    
    #Set lists to add data
    country_type= list() #The type of data Extra/Main as in the web
    country = list() # The country name 
    league = list() # The leagues which covered 
    update_date = list() # last update time of country 
    
    for i in soup.find(text="Main Leagues").findNext('table').find_all('a'):
        
        #Get info for the County type / Country Name/ Leagues
        country_name = str(i.get_text().split()[0])
        Leagues_name = str(i.findNext('td').get_text())           
        country.append(country_name)
        country_type.append("Main")
        league.append(Leagues_name)
        subsoup = get_soup(country_name)
        update_date.append(subsoup.find_all('i')[0].get_text().split()[-1])
    
        
    for i in soup.find(text="Extra Leagues").findNext('table').find_all('a'):
        
        #Get info for the County type / Country Name/ Leagues
        country_name = str(i.get_text().split()[0]) 
        Leagues_name = str(i.findNext('td').get_text())
        country.append(country_name)
        country_type.append("Extra")
        league.append(Leagues_name)
        subsoup = get_soup(country_name)
        update_date.append(subsoup.find_all('i')[0].get_text().split()[-1])
        
    return(set(zip(country_type,country,league,update_date)))

                           


def get_filename(country):   
    soup = get_soup(country)
    result = list()
    for i in soup.find_all('a'):
        txt=i.get('href')
        if txt.find("csv") != -1:
            fileName = i.get('href')
            LeagueName = i.get_text()
            SeasonName = i.findPrevious('i').get_text()
            result.append(country+";"+SeasonName+";"+LeagueName+";"+fileName)
            if len(LeagueName)==0:
                break
    return(result)

        
#get_soup("Mexico")
#get_country(get_soup("data"))
#get_filename("Scotland")


 


In [479]:
soup = get_soup("Brazil")
url = 'https://www.football-data.co.uk/'

In [480]:
for i in soup.find_all('a'):
    txt=i.get('href')
    if txt.find("csv") != -1:
        fileName = i.get('href')
        LeagueName = i.get_text()
        SeasonName = i.findPrevious('i').get_text()
        #if LeagueName=="Division 1" and SeasonName=="Season 2004/2005":
        url_loc = url+fileName
        data = requests.get(url_loc).text
        reader = csv.reader(data.split('\n'), delimiter=',')
        df=pd.DataFrame(reader)
        df.columns = df.iloc[0]
        df=df[1:-1]
        break
            
df.replace('',np.nan,inplace=True)        
df.fillna(np.nan,inplace=True)
names=[i for i in df.columns if i is not None if len(i)>0]
df=df[names]
#df.head(12)
df.rename(columns={"Home":"HomeTeam", "Away": "AwayTeam","HG": "FTHG","AG": "FTAG","Res":"FTR"},inplace=True)
df.head(12)

#df.head(11)

            #df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
            


,Country,League,Season,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,PH,PD,PA,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA
1,Brazil,Serie A,2012,19/05/2012,22:30,Palmeiras,Portuguesa,1,1,D,1.75,3.86,5.25,1.76,3.87,5.31,1.69,3.5,4.9
2,Brazil,Serie A,2012,19/05/2012,22:30,Sport Recife,Flamengo RJ,1,1,D,2.83,3.39,2.68,2.83,3.42,2.7,2.59,3.23,2.58
3,Brazil,Serie A,2012,20/05/2012,01:00,Figueirense,Nautico,2,1,H,1.6,4.04,6.72,1.67,4.05,7.22,1.59,3.67,5.64
4,Brazil,Serie A,2012,20/05/2012,20:00,Botafogo RJ,Sao Paulo,4,2,H,2.49,3.35,3.15,2.49,3.39,3.15,2.35,3.26,2.84
5,Brazil,Serie A,2012,20/05/2012,20:00,Corinthians,Fluminense,0,1,A,1.96,3.53,4.41,1.96,3.53,4.41,1.89,3.33,3.89
6,Brazil,Serie A,2012,20/05/2012,20:00,Internacional,Coritiba,2,0,H,1.76,3.63,5.66,1.77,3.9,6,1.66,3.55,5.1
7,Brazil,Serie A,2012,20/05/2012,20:00,Ponte Preta,Atletico-MG,0,1,A,3.12,3.45,2.44,3.12,3.45,2.62,2.74,3.23,2.45
8,Brazil,Serie A,2012,20/05/2012,22:30,Bahia,Santos,0,0,D,2.33,3.44,3.32,2.45,3.52,3.32,2.3,3.22,2.97
9,Brazil,Serie A,2012,20/05/2012,22:30,Cruzeiro,Atletico GO,0,0,D,1.83,3.74,4.82,1.85,3.79,4.82,1.78,3.47,4.3
10,Brazil,Serie A,2012,20/05/2012,22:30,Vasco,Gremio,2,1,H,2.25,3.4,3.53,2.25,3.46,3.64,2.08,3.25,3.41


In [482]:
tt = pd.read_csv('C:\\Users\\Igor\\Documents\\GitHub\\Foot_ball_data_scrape\\raw_data\\Extra_Brazil.csv')
gg = pd.read_csv('C:\\Users\\Igor\\Documents\\GitHub\\Foot_ball_data_scrape\\raw_data\\Main_Scotland_Division1_Season2004_2005.csv')
gg.head(10)


,country_type,Country,League,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,B365H,B365D,B365A,BWH,BWD,BWA,GBH,GBD,GBA,IWH,IWD,IWA,LBH,LBD,LBA,SBH,SBD,SBA,WHH,WHD,WHA,GBAHH,GBAHA,GBAH
0,Main,Scotland,Division 1,Season 2004/2005,07/08/04,Airdrie Utd,St Johnstone,1,0,H,0,0,D,2.75,3.5,2.10,2.80,3.50,2.10,2.80,3.4,2.10,2.70,3.2,2.1,2.50,3.5,2.25,2.80,3.40,2.10,3.00,3.4,2.0,1.54,2.10,0.5
1,Main,Scotland,Division 1,Season 2004/2005,07/08/04,Clyde,Partick,2,1,H,2,0,H,2.50,3.5,2.25,2.60,3.50,2.20,2.55,3.5,2.25,2.70,3.2,2.1,2.50,3.5,2.25,2.62,3.40,2.20,2.80,3.4,2.1,1.48,2.25,0.5
2,Main,Scotland,Division 1,Season 2004/2005,07/08/04,Hamilton,Raith Rvs,2,0,H,0,0,D,1.90,3.5,3.20,1.80,3.50,3.55,2.00,3.4,3.00,1.90,3.3,3.0,1.91,3.5,3.20,2.00,3.40,3.00,2.00,3.4,3.0,2.00,1.59,-0.5
3,Main,Scotland,Division 1,Season 2004/2005,07/08/04,Queen of Sth,Ross County,0,1,A,0,1,A,2.37,3.5,2.37,2.50,3.50,2.30,2.45,3.3,2.40,2.40,3.1,2.4,2.20,3.5,2.62,2.50,3.25,2.38,2.50,3.4,2.3,1.41,2.40,0.5
4,Main,Scotland,Division 1,Season 2004/2005,07/08/04,St Mirren,Falkirk,2,0,H,0,0,D,2.50,3.4,2.30,2.50,3.50,2.30,2.45,3.3,2.40,2.40,3.1,2.4,2.50,3.5,2.25,2.50,3.40,2.30,2.40,3.4,2.4,1.41,2.40,0.5
5,Main,Scotland,Division 1,Season 2004/2005,14/08/04,Falkirk,Hamilton,1,1,D,0,1,A,1.80,3.5,3.50,1.90,3.50,3.20,1.80,3.4,3.60,1.80,3.3,3.2,1.80,3.5,3.50,1.80,3.40,3.60,1.80,3.5,3.5,1.80,1.75,-0.5
6,Main,Scotland,Division 1,Season 2004/2005,14/08/04,Partick,Airdrie Utd,3,2,H,1,1,D,1.66,3.6,4.00,1.75,3.60,3.65,1.75,3.4,3.85,1.80,3.3,3.2,1.67,3.6,4.00,1.73,3.40,4.00,1.72,3.4,4.0,1.75,1.81,-0.5
7,Main,Scotland,Division 1,Season 2004/2005,14/08/04,Raith Rvs,Clyde,2,3,A,0,2,A,2.50,3.5,2.25,2.50,3.45,2.30,2.60,3.4,2.25,2.60,3.3,2.1,2.62,3.5,2.20,2.62,3.40,2.20,2.75,3.5,2.1,1.47,2.25,0.5
8,Main,Scotland,Division 1,Season 2004/2005,14/08/04,Ross County,St Mirren,1,1,D,1,0,H,2.00,3.4,3.00,2.10,3.50,2.75,2.00,3.4,3.00,1.90,3.2,3.1,1.80,3.5,3.50,1.91,3.50,3.20,1.80,3.6,3.4,2.00,1.59,-0.5
9,Main,Scotland,Division 1,Season 2004/2005,14/08/04,St Johnstone,Queen of Sth,1,3,A,0,2,A,1.61,3.6,4.33,1.55,3.75,4.75,1.67,3.5,4.20,1.65,3.4,3.8,1.67,3.6,4.00,1.67,3.60,4.00,1.66,3.6,4.0,1.67,1.91,-0.5


In [483]:
tt.head(10)

,country_type,Country,League,Season,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,PH,PD,PA,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA
0,Extra,Brazil,Serie A,2012,19/05/2012,22:30,Palmeiras,Portuguesa,1,1,D,1.75,3.86,5.25,1.76,3.87,5.31,1.69,3.50,4.90
1,Extra,Brazil,Serie A,2012,19/05/2012,22:30,Sport Recife,Flamengo RJ,1,1,D,2.83,3.39,2.68,2.83,3.42,2.70,2.59,3.23,2.58
2,Extra,Brazil,Serie A,2012,20/05/2012,01:00,Figueirense,Nautico,2,1,H,1.60,4.04,6.72,1.67,4.05,7.22,1.59,3.67,5.64
3,Extra,Brazil,Serie A,2012,20/05/2012,20:00,Botafogo RJ,Sao Paulo,4,2,H,2.49,3.35,3.15,2.49,3.39,3.15,2.35,3.26,2.84
4,Extra,Brazil,Serie A,2012,20/05/2012,20:00,Corinthians,Fluminense,0,1,A,1.96,3.53,4.41,1.96,3.53,4.41,1.89,3.33,3.89
5,Extra,Brazil,Serie A,2012,20/05/2012,20:00,Internacional,Coritiba,2,0,H,1.76,3.63,5.66,1.77,3.90,6.00,1.66,3.55,5.10
6,Extra,Brazil,Serie A,2012,20/05/2012,20:00,Ponte Preta,Atletico-MG,0,1,A,3.12,3.45,2.44,3.12,3.45,2.62,2.74,3.23,2.45
7,Extra,Brazil,Serie A,2012,20/05/2012,22:30,Bahia,Santos,0,0,D,2.33,3.44,3.32,2.45,3.52,3.32,2.30,3.22,2.97
8,Extra,Brazil,Serie A,2012,20/05/2012,22:30,Cruzeiro,Atletico GO,0,0,D,1.83,3.74,4.82,1.85,3.79,4.82,1.78,3.47,4.30
9,Extra,Brazil,Serie A,2012,20/05/2012,22:30,Vasco,Gremio,2,1,H,2.25,3.40,3.53,2.25,3.46,3.64,2.08,3.25,3.41


In [489]:
re = pd.concat([tt,gg], axis=0, ignore_index=True,sort=True)
re.to_csv('C:\\Users\\Igor\\Documents\\GitHub\\Foot_ball_data_scrape\\raw_data\\test.csv'+csvfileName,index=False)

In [14]:
subdata = get_country(get_soup("data"))


In [16]:
for i in subdata:
    print(i)

('Main', 'Portugal', 'Liga I', '07/10/19')
('Extra', 'Austria', 'Bundesliga', '08/10/19')
('Main', 'Italy', 'Serie A & B', '07/10/19')
('Extra', 'Japan', 'J-League', '08/10/19')
('Main', 'England', 'Premiership & Divs 1,2,3 & Conference', '09/10/19')
('Main', 'Belgium', 'Jupiler League', '07/10/19')
('Main', 'Germany', 'Bundesligas 1 & 2', '08/10/19')
('Extra', 'Argentina', 'Primera Division', '08/10/19')
('Extra', 'Mexico', 'Liga MX', '08/10/19')
('Extra', 'Norway', 'Eliteserien', '08/10/19')
('Main', 'Scotland', 'Premiership & Divs 1,2 & 3', '07/10/19')
('Extra', 'Denmark', 'Superliga', '08/10/19')
('Extra', 'Sweden', 'Allsvenskan', '08/10/19')
('Main', 'Netherlands', 'KPN Eredivisie', '07/10/19')
('Extra', 'Poland', 'Ekstraklasa', '08/10/19')
('Main', 'Turkey', 'Ligi 1', '07/10/19')
('Extra', 'Russia', 'Premier League', '08/10/19')
('Main', 'Greece', 'Ethniki Katigoria', '08/10/19')
('Main', 'Spain', 'La Liga (Premera & Segunda)', '07/10/19')
('Extra', 'Romania', 'Liga 1', '08/10/19

In [ ]:

for i in subdata:
    
    print(i)
    country_type= i[0] #The type of data Extra/Main as in the web
    country = i[1] # The country name 
    league = i[2] # The leagues which covered 
    update_date =  i[3] # last update time of country
    
    url = 'https://www.football-data.co.uk/'
    soup = get_soup(country)
    for i in soup.find_all('a'):
        txt=i.get('href')
        if txt.find("csv") != -1:
            fileName = i.get('href')
            LeagueName = i.get_text()
            SeasonName = i.findPrevious('i').get_text()
            url_loc = url+fileName
            data = requests.get(url_loc).text
            reader = csv.reader(data.split('\n'), delimiter=',')
            
            df=pd.DataFrame(reader)
            df.columns = df.iloc[0]
            df=df[1:-1]
            df.replace('',999,inplace=True)        
            df.fillna(999,inplace=True)  
            names=[i for i in df.columns if i is not None if len(i)>0]
            df=df[names]
            df.rename(columns = {"Home":"HomeTeam", "Away": "AwayTeam","HG": "FTHG","AG": "FTAG","Res":"FTR"},inplace=True)
            
            print(country+"===="+SeasonName+"===="+LeagueName)
            
            if len(LeagueName)==0:
                
                df.insert(0, 'country_type', country_type)
                csvfileName = country_type+"_"+country+".csv"
                csvfileName = csvfileName.replace(" ","").replace("/","_")             
                df.to_csv('C:\\Users\\Igor\\Documents\\GitHub\\Foot_ball_data_scrape\\raw_data\\'+csvfileName,index=False)
                break
                
            else:
                df.drop('Div', axis=1, inplace=True)
                df.insert(loc=0, column='country_type', value=country_type)
                df.insert(loc=1, column='Country', value=country)
                df.insert(loc=2, column='League', value=LeagueName)
                df.insert(loc=3, column='Season', value=SeasonName.split()[1])
                csvfileName = country_type+"_"+country+"_"+LeagueName+"_"+SeasonName+".csv"
                csvfileName = csvfileName.replace(" ","").replace("/","_")
                df.to_csv('C:\\Users\\Igor\\Documents\\GitHub\\Foot_ball_data_scrape\\raw_data\\'+csvfileName,index=False)
           
    
    


('Main', 'Portugal', 'Liga I', '07/10/19')
Portugal====Season 2019/2020====Liga I
Portugal====Season 2018/2019====Liga I
Portugal====Season 2017/2018====Liga I
Portugal====Season 2016/2017====Liga I
Portugal====Season 2015/2016====Liga I
Portugal====Season 2014/2015====Liga I
Portugal====Season 2013/2014====Liga I
Portugal====Season 2012/2013====Liga I
Portugal====Season 2011/2012====Liga I
Portugal====Season 2010/2011====Liga I
Portugal====Season 2009/2010====Liga I
Portugal====Season 2008/2009====Liga I
Portugal====Season 2007/2008====Liga I
Portugal====Season 2006/2007====Liga I
Portugal====Season 2005/2006====Liga I
Portugal====Season 2004/2005====Liga I
Portugal====Season 2003/2004====Liga I
Portugal====Season 2002/2003====Liga I
Portugal====Season 2001/2002====Liga I
Portugal====Season 2000/2001====Liga I
Portugal====Season 1999/2000====Liga I
Portugal====Season 1998/1999====Liga I
Portugal====Season 1997/1998====Liga I
Portugal====Season 1996/1997====Liga I
Portugal====Season 19